In [1]:
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os

In [3]:
train_df = pd.read_csv('data/train.csv', usecols = ['Cliente_ID',
       'Producto_ID','Demanda_uni_equil' ])

In [5]:
train_df.shape

(74180464, 3)

In [7]:
train_df['log1p'] = np.log1p(train_df.Demanda_uni_equil)

In [9]:
print ('In train_df.csv, there are: ')
for col in train_df.columns:
     print ('{:>10}{:>20}'.format(len(train_df[col].value_counts()), col))

In train_df.csv, there are: 
    880604          Cliente_ID
      1799         Producto_ID
      2091   Demanda_uni_equil
      2091               log1p


In [10]:
train_df.head(3)

,Cliente_ID,Producto_ID,Demanda_uni_equil,log1p
0,15766,1212,3,1.386294
1,15766,1216,4,1.609438
2,15766,1238,4,1.609438


In [13]:
log1p_mean_df = train_df.groupby(['Cliente_ID','Producto_ID'], as_index=False).mean()
log1p_mean_df['log1p_mean'] = np.expm1(log1p_mean_df.log1p)
log1p_mean_df.drop(['log1p','Demanda_uni_equil'] , axis = 1, inplace = True)
global_mean = log1p_mean_df.log1p_mean.mean()

In [14]:
print ('Global log1p_mean of demanda:', global_mean)
log1p_mean_df.shape

Global log1p_mean of demanda: 5.182339442038257


(25831538, 3)

In [15]:
log1p_mean_df.head(4)

,Cliente_ID,Producto_ID,log1p_mean
0,26,1182,2.419952
1,26,4767,42.000000
2,26,30235,96.000000
3,26,30314,48.000000


In [16]:
log1p_mean_df.rename(columns = {'log1p_mean':'Demanda_uni_equil'}, inplace = True)

In [21]:
check = train_df[train_df.Cliente_ID == 2025]
check.head()

,Cliente_ID,Producto_ID,Demanda_uni_equil,log1p
7500453,2025,1109,4,1.609438
7500454,2025,1125,8,2.197225
7500455,2025,1146,4,1.609438
7500456,2025,1150,10,2.397895
7500457,2025,1160,5,1.791759


In [18]:
test_df = pd.read_csv('data/test.csv', usecols=['Cliente_ID', 'Producto_ID'])

In [23]:
print(test_df.shape)
test_df.head()

(6999251, 2)


,Cliente_ID,Producto_ID
0,4639078,35305
1,4705135,1238
2,4549769,32940
3,4717855,43066
4,966351,1277


In [24]:
results_df = test_df.merge(log1p_mean_df, on = ['Cliente_ID','Producto_ID'], how='left')

In [26]:
results_df.head()

,Cliente_ID,Producto_ID,Demanda_uni_equil
0,4639078,35305,4.0
1,4705135,1238,NaN
2,4549769,32940,2.0
3,4717855,43066,NaN
4,966351,1277,NaN


In [28]:
results_df.fillna(global_mean, inplace =True)

In [29]:
results_df.head(3)

,Cliente_ID,Producto_ID,Demanda_uni_equil
0,4639078,35305,4.000000
1,4705135,1238,5.182339
2,4549769,32940,2.000000


In [30]:
results_df.drop(['Cliente_ID','Producto_ID'], axis = 1, inplace = True)

In [27]:
results_df.to_csv('data/submission/log1p_mean_submission.csv', index_label=['id'])